In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
import matplotlib

In [ ]:
# precision-recall curve and f1
from matplotlib import pyplot
from scipy.stats import chi2, norm


In [ ]:
%matplotlib inline

In [ ]:
plt.rc('font', size=12)

In [ ]:
# confidence limits for 1, 2, and 3 standard deviations in 1 dimension
nstd1 = 2. * (norm.cdf(1) - 0.5)
nstd2 = 2. * (norm.cdf(2) - 0.5)
nstd3 = 2. * (norm.cdf(3) - 0.5)
#print (nstd1, nstd2, nstd3)

# confidence limits in two dimensions
# 68.3% = 1 std dev (1 dim)
l68 = chi2.ppf(nstd1, 2)
# 95.4% = 2 std dev (1 dim)
l95 = chi2.ppf(nstd2, 2)
# 99.7% = 3 std dev (1 dim)
l99 = chi2.ppf(nstd3, 2)

# scales with which to scale up r1 and r2
scale1 = np.sqrt(l68)
scale2 = np.sqrt(l95)
scale3 = np.sqrt(l99)
#print (scale1, scale2, scale3)

print (l68, l95, l99)

In [ ]:
from scipy.special import xlogy

def phat(rec, prec, x_tp, x_fp, x_tn, x_fn):
    """Fit probability parameters of confusion matrix under the constraint of 
    fixed recall and precision
    """
    n4 = x_tp + x_fp + x_tn + x_fn
    n3 = x_tp + x_fp + x_fn
    alpha = (1-prec)/prec + (1-rec)/rec + 1
    p_tp = (n3 / n4) * (1. / alpha)
    p_fn = ((1-rec)/rec) * p_tp
    p_fp = ((1-prec)/prec) * p_tp
    p_tn = 1. - p_fn - p_fp - p_tp 
    if isinstance(p_tn, np.ndarray):
        p_tn[p_tn < 0] = 0
    elif isinstance(p_tn, float) and p_tn < 0:
        p_tn = 0.
    
    return p_tp, p_fp, p_tn, p_fn

def nll(rec, prec, x_tp, x_fp, x_tn, x_fn):
    """Return -2logp of multinomial distribution
    
    1. Fit with fixed recall and precision 
    2. Fit with all probability parameters free
    
    Return the difference in -2 log L
    """
    p_tp, p_fp, p_tn, p_fn = phat(rec, prec, x_tp, x_fp, x_tn, x_fn)    
    nll_value = -2 * xlogy(x_tp, p_tp) - 2 * xlogy(x_fp, p_fp) - 2 * xlogy(x_fn, p_fn) - 2 * xlogy(x_tn, p_tn)

    n4 = x_tp + x_fp + x_tn + x_fn
    p_fn0 = x_fn / n4
    p_tp0 = x_tp / n4
    p_fp0 = x_fp / n4
    p_tn0 = x_tn / n4
    nll_minimum = -2 * xlogy(x_tp, p_tp0) - 2 * xlogy(x_fp, p_fp0) - 2 * xlogy(x_fn, p_fn0) - 2 * xlogy(x_tn, p_tn0)    

    return nll_value - nll_minimum

In [ ]:
# confusion matrix
x_tp = 100
x_fp = 10
x_tn = 1620
x_fn = 100

# how fine-grained is the precision-recall grid?
# MC simulations
nbins1 = 40
# PLL calculation
nbins2 = 500

# number of pseudo experiments per R,P point
n_toys = 50000

In [ ]:
N = x_tp + x_fp + x_tn + x_fn

In [ ]:
rec = x_tp / (x_tp + x_fn)
prec = x_tp / (x_tp + x_fp)

In [ ]:
rec, prec, N

In [ ]:
# Next: make a rough estimate for the range of the precision-recall grid

In [ ]:
if rec == 0:
    rec_for_sigma = 1 / (x_tp + x_fn)
elif rec == 1:
    rec_for_sigma = (x_tp + x_fn - 1) / (x_tp + x_fn)
else:
    rec_for_sigma = rec
    
if prec == 0:
    prec_for_sigma = 1 / (x_tp + x_fp)
elif prec == 1:
    prec_for_sigma = (x_tp + x_fp - 1) / (x_tp + x_fp)
else:
    prec_for_sigma = prec

In [ ]:
sigma_rec = np.sqrt((rec_for_sigma*(1-rec_for_sigma))/(x_tp + x_fn))
sigma_prec = np.sqrt((prec_for_sigma*(1-prec_for_sigma))/(x_tp + x_fp))

In [ ]:
# epsilon to prevent division by zero at edge
epsilon = 1e-4

rec_max = min(rec + 6 * sigma_rec, 1)
rec_min = max(rec - 7 * sigma_rec, epsilon)

prec_max = min(prec + 6 * sigma_prec, 1)
prec_min = max(prec - 7 * sigma_prec, epsilon)

# the plot range
rec_min_range = max(rec - 7 * sigma_rec, 0)
prec_min_range = max(prec - 7 * sigma_prec, 0)

In [ ]:
(rec_min, rec_max), (prec_min, prec_max)

In [ ]:
# PR grid for MC simulations

In [ ]:
rx = np.linspace(rec_min, rec_max, nbins1)
py = np.linspace(prec_min, prec_max, nbins1)
RX, PY = np.meshgrid(rx, py)

In [ ]:
P_TP, P_FP, P_TN, P_FN = phat(RX, PY, x_tp, x_fp, x_tn, x_fn)

In [ ]:
the_shape = RX.shape

RX = RX.ravel()
PY = PY.ravel()
P_TP = P_TP.ravel()
P_FP = P_FP.ravel()
P_TN = P_TN.ravel()
P_FN = P_FN.ravel()

In [ ]:
# size of the grid
nbins1 ** 2

In [ ]:
# run MC simulations
# evaluate coverage for each precision, recall point.
covg = []

for r_x, p_y, p_tp, p_fp, p_tn, p_fn in tqdm(zip(RX, PY, P_TP, P_FP, P_TN, P_FN)):
    ph = [p_tp, p_fp, p_tn, p_fn]
    X = np.random.multinomial(N, ph, n_toys)
    X_TP = X[:, 0]
    X_FP = X[:, 1]
    X_TN = X[:, 2]
    X_FN = X[:, 3]
    dchi2 = nll(r_x, p_y, X_TP, X_FP, X_TN, X_FN)
    dchi2_null = nll(r_x, p_y, x_tp, x_fp, x_tn, x_fn)
    coverage = np.sum(dchi2 < dchi2_null) / n_toys
    # print (r_x, p_y, coverage)
    covg.append(coverage)

covg = np.array(covg)

In [ ]:
RX = RX.reshape(the_shape)
PY = PY.reshape(the_shape)
covg = covg.reshape(the_shape)

In [ ]:
#chi2.cdf(dchi2_null, 2)

In [ ]:
# evaluate analytical coverage for each precision, recall point.

rx = np.linspace(rec_min, rec_max, nbins2)
py = np.linspace(prec_min, prec_max, nbins2)
RX2, PY2 = np.meshgrid(rx, py)
Z = nll(RX2, PY2, x_tp, x_fp, x_tn, x_fn)

In [ ]:
# plot both sets of contours
fig, ax = plt.subplots(figsize=(12,7))

CS = ax.contour(RX2, PY2, Z, levels=[l68, l95, l99])
CS = ax.contour(RX, PY, covg, levels=[nstd1, nstd2, nstd3])
ax.clabel(CS, inline=True, fontsize=10)
ax.set_xlim(rec_min_range, rec_max)
ax.set_ylim(prec_min_range, prec_max)

title = f'TP: {x_tp:.1f}, FP: {x_fp:.1f}, FN: {x_fn:.1f}, test size: {N}'
ax.set_title(title)
ax.grid()
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

ax.plot(rec, prec,'ro') 

plt.tight_layout()
plt.savefig(f'PR_exclusion_contours_{x_fp:.1f}FP.pdf')